In [1]:
pip install sqlalchemy


Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
from sqlalchemy import create_engine
import getpass
password = getpass.getpass()

bd ="sakila"
connection_string = 'mysql+pymysql://root:' + password + '@localhost/'+bd
engine = create_engine(connection_string)
engine

def rentals_month(engine, month, year):
    """
    Retrieve rental data from the rental table for a specified month and year.
    
    Parameters:
    - engine: SQLAlchemy engine object to connect to the database.
    - month: Integer representing the month.
    - year: Integer representing the year.
    
    Returns:
    - DataFrame containing rental data for the specified month and year.
    """
    query = f"""
    SELECT rental_id, rental_date, customer_id
    FROM rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    """
    df = pd.read_sql(query, engine)
    return df

# 2. Function to count rentals per customer for a specific month and year
def rental_count_month(df, month, year):
    """
    Count the number of rentals per customer for a specific month and year.
    
    Parameters:
    - df: DataFrame containing rental data.
    - month: Integer representing the month.
    - year: Integer representing the year.
    
    Returns:
    - DataFrame with customer_id and a column named 'rentals_<month>_<year>' 
      containing the number of rentals for each customer.
    """
    rental_counts = df.groupby('customer_id')['rental_id'].count().reset_index()
    rental_counts.columns = ['customer_id', f'rentals_{month:02}_{year}']
    return rental_counts

# 3. Function to compare rental data for two different months
def compare_rentals(df1, df2):
    """
    Compare the number of rentals between two months.
    
    Parameters:
    - df1: DataFrame containing rental counts for the first month.
    - df2: DataFrame containing rental counts for the second month.
    
    Returns:
    - A merged DataFrame with customer_id, rental counts for both months, and the difference in rentals.
    """
    # Merging two dataframes on 'customer_id' (outer join to include all customers)
    merged_df = pd.merge(df1, df2, on='customer_id', how='outer').fillna(0)
    
    # Extracting the rental columns
    rental_columns = [col for col in merged_df.columns if 'rentals' in col]
    
    # Calculating the difference between the two months
    merged_df['difference'] = merged_df[rental_columns[0]] - merged_df[rental_columns[1]]
    
    return merged_df

# Example usage:

# 1. Retrieve rental data for May and June 2005
may_rentals = rentals_month(engine, 5, 2005)
june_rentals = rentals_month(engine, 6, 2005)

# 2. Count the number of rentals per customer for May and June 2005
may_rental_count = rental_count_month(may_rentals, 5, 2005)
june_rental_count = rental_count_month(june_rentals, 6, 2005)

# 3. Compare rentals between May and June 2005
comparison = compare_rentals(may_rental_count, june_rental_count)

# 4. Print the comparison DataFrame
print(comparison)

     customer_id  rentals_05_2005  rentals_06_2005  difference
0              1              2.0              7.0        -5.0
1              2              1.0              1.0         0.0
2              3              2.0              4.0        -2.0
3              4              0.0              6.0        -6.0
4              5              3.0              5.0        -2.0
..           ...              ...              ...         ...
593          595              1.0              2.0        -1.0
594          596              6.0              2.0         4.0
595          597              2.0              3.0        -1.0
596          598              0.0              1.0        -1.0
597          599              1.0              4.0        -3.0

[598 rows x 4 columns]
